In [70]:
import psycopg2
from psycopg2 import DatabaseError

In [71]:
import datetime
from datetime import date

In [72]:
import names
import string
import random

In [73]:
client_name = "John Smith"
amount = 700
ticket_price = 250

In [74]:
conn_fly_string = "host='localhost' dbname='flyDB2' user='postgres' password='postgres'"
conn_hotel_string = "host='localhost' dbname='hotelDB2' user='postgres' password='postgres'"
conn_account_string = "host='localhost' dbname='accountDB2' user='postgres' password='postgres'"

In [75]:
try:
    conn_fly = psycopg2.connect(conn_fly_string)
except:
    print "I am unable to connect to the database fly"

In [76]:
try:
    conn_hotel = psycopg2.connect(conn_hotel_string)
except:
    print "I am unable to connect to the database hotel"

In [77]:
try:
    conn_account = psycopg2.connect(conn_account_string)
except:
    print "I am unable to connect to the database account"

In [78]:
cursor_fly = conn_fly.cursor()
cursor_hotel = conn_hotel.cursor()
cursor_account = conn_account.cursor()

In [79]:
statement = """
CREATE TABLE IF NOT EXISTS  public.fly
(
    "id"  serial PRIMARY KEY,
    "clientname" character varying(50) COLLATE pg_catalog."default",
    "flynumber" character varying(16) COLLATE pg_catalog."default",
    frm character varying(8) COLLATE pg_catalog."default",
    "to_" character varying(8) COLLATE pg_catalog."default",
    date date
)
;"""

In [80]:
cursor_fly.execute(statement)

In [81]:
conn_fly.commit()

In [82]:
statement = """
CREATE TABLE IF NOT EXISTS public.hotel
(
    "id"  serial PRIMARY KEY,
    "clientname" character varying(50) COLLATE pg_catalog."default",
    "hotelname" character varying(50) COLLATE pg_catalog."default",
    arrival date,
    departure date
)
;"""

In [83]:
cursor_hotel.execute(statement)

In [84]:
conn_hotel.commit()

In [85]:
statement = """
CREATE TABLE IF NOT EXISTS account
(
    "id" serial PRIMARY KEY,
    "clientname" character varying(50) COLLATE pg_catalog."default",
    amount integer,
    CONSTRAINT ck_account_amount_gt_0 CHECK (amount > 0)
)
;"""

In [86]:
cursor_account.execute(statement)

In [87]:
conn_account.commit()

In [88]:
cursor_account.execute("INSERT INTO account(clientname, amount) VALUES (%s, %s) RETURNING id", (client_name, amount) )

In [89]:
account_id = cursor_account.fetchone()[0]

In [90]:
conn_account.commit()

In [91]:
account_id

1

In [92]:
def getRandomFlyNumber(sizeAscii=3, sizeDigits=4,chars=string.ascii_uppercase, digits=string.digits):
    return ''.join(random.choice(chars) for x in range(sizeAscii)) + '-' + ''.join(random.choice(digits) for x in range(sizeDigits))

In [93]:
def getRandomAirport(sizeAscii=3,chars=string.ascii_uppercase):
    return ''.join(random.choice(chars) for x in range(sizeAscii))

In [94]:
def getRandomeDate():
    start_date = date.today().replace(day=1, month=1).toordinal()
    end_date = date.today().toordinal()
    random_day = date.fromordinal(random.randint(start_date, end_date))
    return random_day

In [95]:
def booking(id, flynumber, from_, to, date, hotel, arrival, departure, total):

    # begin transactions for each database connection
    conn_fly.tpc_begin(conn_fly.xid(43, 'transaction ID', 'connection fly'))
    conn_hotel.tpc_begin(conn_hotel.xid(43, 'transaction ID', 'connection hotel'))
    conn_account.tpc_begin(conn_account.xid(43, 'transaction ID', 'connection account'))


    
    try:
        
        # Do stuff with all connections
        # todo
    
        cursor_fly.execute(
            "INSERT INTO fly (clientname, flynumber, frm, to_, date) "
            "VALUES ( %(name)s, %(number)s, %(frm)s, %(to)s, %(date)s )", {'name': client_name, 'number':flynumber, 'frm':from_, 'to': to, 'date':date})
        conn_fly.tpc_prepare()

        cursor_hotel.execute(
            "INSERT INTO hotel (clientname, hotelname, arrival, departure) "
            "VALUES ( %(name)s, %(hotel)s, %(arrival)s, %(departure)s)", {'name': client_name, 'hotel':flynumber, 'arrival':arrival, 'departure': departure})
        conn_hotel.tpc_prepare()
    
        sql = """ UPDATE account
                SET amount = amount - %s
                WHERE id = %s"""
        cursor_account.execute(
            sql, (total, id))    
        conn_account.tpc_prepare()
        
    except DatabaseError :
        conn_fly.tpc_rollback()
        conn_hotel.tpc_rollback()
        conn_account.tpc_rollback()
    else:
        conn_fly.tpc_commit()
        conn_hotel.tpc_commit()
        conn_account.tpc_commit()

In [96]:
# first booking
booking(id=account_id, flynumber=getRandomFlyNumber(), from_=getRandomAirport(), to=getRandomAirport(), date=getRandomeDate(), hotel=names.get_first_name(), arrival=getRandomeDate(), departure=getRandomeDate(), total = ticket_price)


In [97]:
# second booking
booking(id=account_id, flynumber=getRandomFlyNumber(), from_=getRandomAirport(), to=getRandomAirport(), date=getRandomeDate(), hotel=names.get_first_name(), arrival=getRandomeDate(), departure=getRandomeDate(), total = ticket_price)

In [98]:
# third booking
booking(id=account_id, flynumber=getRandomFlyNumber(), from_=getRandomAirport(), to=getRandomAirport(), date=getRandomeDate(), hotel=names.get_first_name(), arrival=getRandomeDate(), departure=getRandomeDate(), total = ticket_price)